In [1]:
import numpy as np
import h5py
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D,Flatten,MaxPooling2D,ZeroPadding2D,Concatenate,Lambda,Softmax,GlobalAveragePooling1D,MaxPooling1D,SpatialDropout1D,ReLU, Dense, Activation,Reshape,BatchNormalization, add, Embedding,Conv1D,LayerNormalization,MultiHeadAttention,Add,Dropout,Layer
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau, EarlyStopping
from Uilts import *
from capsulelayers import *

In [2]:
import h5py
import numpy as np
import tensorflow as tf

# ========== Load dataset from HDF5 file ==========
# This file contains two datasets:
# - 'datall': the input data (e.g., scalograms)
# - 'laballx': the corresponding labels (0 for EQ, 1 for QB)

with h5py.File('./Dataset/data_train.h5', 'r') as hf:
    datall = hf['datall'][:]     # Load all input data into memory
    laballx = hf['laballx'][:]   # Load all labels into memory

# ========== Print dataset shapes ==========
print("datall shape:", datall.shape)   
print("laballx shape:", laballx.shape) 

# ========== Convert integer labels to one-hot encoding ==========
# Converts labels:
#   0 → [1, 0]  (Earthquake)
#   1 → [0, 1]  (Quarry Blast)
laball = tf.keras.utils.to_categorical(laballx, num_classes=2)

# ========== Print label distribution ==========
print('Earthquakes:',len(np.where(laballx == 0)[0]), 'Quarry blasts:',len(np.where(laballx == 1)[0]))  # Count EQ and QB samples


datall shape: (6444, 20, 6000, 3)
laballx shape: (6444,)
Earthquakes: 3169 Quarry blasts: 3275


In [3]:
# Split dataset
from sklearn.model_selection import train_test_split
# np.random.seed(42)
# io = np.random.permutation(len(datall))
# datall = datall[io]
# laball = laball[io]

sp = 0.1
x_train, x_test, y_train, y_test = train_test_split(datall, laball, test_size=sp, random_state=2024)
print(x_train.shape)
print(y_train.shape)

(5799, 20, 6000, 3)
(5799, 2)


In [4]:
# Hyperparameters of the network

w1 = 20
w2 = 6000

learning_rate = 0.001
weight_decay = 0.0001
stochastic_depth_rate = 0.1
positional_emb = False
conv_layers = 6
num_classes = 1
input_shape = (w1, w2, 3)
image_size = w1  # We'll resize input images to this size
projection_dim = int(w1)
num_heads = 3
transformer_units = [
    projection_dim,
    projection_dim,
]  # Number of the transformer layers
transformer_layers = 1
mlp_head_units = [
    1024,
    512,
]

image_size = 6000
patch_size = 20
num_patches = (image_size // patch_size)

In [5]:
# ========= Transformer-based Feature Extractor =========
# Required: Custom components such as DCBlock, Patches, PatchEncoder, mlp, StochasticDepth
def create_cct_model(inputs):
    """
    Feature extractor using Dilated Convolution Blocks and Transformer layers.
    """
    # Apply multiple dilated convolutional blocks
    x = DCBlock(inputs, 6, (3, 3), 0.1)
    x = DCBlock(x, 12, (3, 3), 0.1)
    x = DCBlock(x, 20, (3, 3), 0.1)

    # Patch embedding
    patches = Patches(patch_size)(x)
    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)

    # Define stochastic depth rates for each Transformer layer
    dpr = np.linspace(0, stochastic_depth_rate, transformer_layers)

    # Stack multiple Transformer blocks
    for i in range(transformer_layers):
        # Layer normalization
        x1 = LayerNormalization(epsilon=1e-5)(encoded_patches)

        # Multi-head self-attention
        attn_output = MultiHeadAttention(
            num_heads=num_heads, key_dim=projection_dim, dropout=0.1
        )(x1, x1)

        # Skip connection and stochastic depth
        attn_output = StochasticDepth(dpr[i])(attn_output)
        x2 = Add()([attn_output, encoded_patches])

        # Second layer normalization
        x3 = LayerNormalization(epsilon=1e-5)(x2)

        # Feed-forward MLP with dropout
        x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=0.2)

        # Skip connection and stochastic depth
        x3 = StochasticDepth(dpr[i])(x3)
        encoded_patches = Add()([x3, x2])

    # Final layer normalization
    representation = LayerNormalization(epsilon=1e-5)(encoded_patches)
    return representation


# ========= Capsule Network Model =========

def QBTransNet(input_shape, n_class, routings):
    """
    Capsule Network with Transformer-based encoder.
    
    Parameters:
    - input_shape: shape of input data [height, width, channels]
    - n_class: number of target classes
    - routings: number of dynamic routing iterations
    
    Returns:
    - Compiled Keras model
    """
    capsule_dim = 4
    inputs = Input(shape=input_shape)

    # Transformer feature extractor
    conv_features = create_cct_model(inputs)

    # Reshape encoded sequence into spatial format
    conv_features = Reshape((20, 15, 20))(conv_features)

    # Primary capsule layer: extracts low-level capsules
    primarycaps = PrimaryCap(
        conv_features, dim_capsule=capsule_dim, n_channels=4,
        kernel_size=5, strides=(1, 4), padding='same'
    )
    primarycaps = Dropout(0.5)(primarycaps)

    # Digit capsule layer: high-level capsules with dynamic routing
    digitcaps = CapsuleLayer(
        num_capsule=n_class, dim_capsule=capsule_dim,
        routings=routings, name='digitcaps'
    )(primarycaps)

    # Output capsule length as probability
    out_caps = Length(name='capsnet')(digitcaps)

    # Build and compile model
    model = Model(inputs=inputs, outputs=out_caps)
    model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
                  loss=[margin_loss], metrics=['acc'])

    model.summary()
    return model

def run_experiment(model):
    """
    Train the provided model with training data and specified callbacks.

    Returns:
        history: Training history object containing loss and accuracy metrics.
    """

    # Callback to reduce learning rate when validation accuracy plateaus
    lr_reducer = ReduceLROnPlateau(
        factor=np.sqrt(0.1),       # Reduce LR by sqrt(0.1)
        cooldown=0,                # No cooldown period before resuming LR adjustments
        patience=20,               # Wait 20 epochs before reducing LR
        min_lr=0.5e-8,             # Minimum allowable LR
        monitor='val_acc',         # Monitor validation accuracy
        mode='max'                 # Looking for maximum accuracy
    )

    # Callback to stop training early if validation accuracy doesn't improve
    early_stopping_monitor = EarlyStopping(
        monitor='val_acc',
        mode='max',
        patience=30                # Stop training after 30 epochs of no improvement
    )

    # Callback to save the best model weights during training
    checkpoint = ModelCheckpoint(
        filepath='Epochs/best_model.h5',
        monitor='val_acc',        # Monitor validation accuracy
        mode='max',
        save_best_only=True,      # Save only the best model
        verbose=1                 # Print updates when model is saved
    )

    # Combine all callbacks into a list
    callbacks = [lr_reducer, early_stopping_monitor, checkpoint]

    # Train the model with training data and 10% validation split
    history = model.fit(
        x_train, y_train,
        epochs=200,
        validation_split=0.1,
        batch_size=16,
        callbacks=callbacks
    )

    return history


In [6]:
import time

if __name__ == '__main__':
    start_time = time.time()
    create = QBTransNet(input_shape=(20,6000,3), n_class=2, routings=3)
    history = run_experiment(create)
    end_time = time.time()

    # Calculate training time
    training_time = end_time - start_time
    print("Training time is：", training_time, "s")

Instructions for updating:
Use fn_output_signature instead
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 20, 6000, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 20, 6000, 3)  84          ['input_1[0][0]']                
                                                                                                  
 activation (Activation)        (None, 20, 6000, 3)  0           ['conv2d[0][0]']                 
                                                                                                  
 conv2d_1 (Conv2D)              (No